In [12]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [11]:
import pyswarms
from pyswarms.single.global_best import GlobalBestPSO

In [3]:
data=pd.read_csv("Bank_Personal_Loan_Modelling.csv")
y=data["Personal Loan"]
data["Loan"]=y
data.drop(columns="Personal Loan",inplace=True)

In [4]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()
Y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [23]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=2)

In [24]:
def create_custom_model(nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=13, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


In [28]:
n_layers = 1
model = create_custom_model(4, n_layers)
model.summary()

score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 4)                 56        
                                                                 
 dense_7 (Dense)             (None, 2)                 10        
                                                                 
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________
79/79 [==============================] - 0s 1ms/step - loss: 0.7448 - accuracy: 0.5040
Test loss: 0.7447980046272278
Test accuracy: 0.5040000081062317


In [19]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [29]:
def evaluate_nn(W, shape,X_train=X_train, Y_train=Y_train):
    results = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_train, Y_train, verbose=0)
        results.append(1-score[1])
    return results

shape = get_shape(model)
x_max = 1.0 * np.ones(66)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.6, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=50, dimensions=66,
                          options=options, bounds=bounds)
cost, pos = optimizer.optimize(evaluate_nn, 20, X_train=X_train, Y_train=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2023-04-04 20:07:49,735 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.4, 'c2': 0.6, 'w': 0.4}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|20/20, best_cost=0.0632
2023-04-04 20:11:05,715 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.06319999694824219, best pos: [ 0.26649501  0.04011821  0.36470057 -0.25069635  0.1011955   0.20887863
 -0.08957668 -0.10849412  0.46163856  0.17397984  0.55134475  0.16293171
 -0.11753534  0.64927061 -0.25916789  0.18032187 -0.07410538  0.19772672
  0.07088687  0.41801056  0.3225276   0.56655185  0.39869114  0.13889542
  0.17763718  0.339105   -0.19892247  0.0657732  -0.03661949  0.24621249
 -0.06935342  0.38054783 -0.22264136  0.26457824  0.1813031   0.18981597
 -0.03262387 -0.23268254  0.09953545 -0.16634124  0.06612926  0.02633322
  0.13128012  0.73194432  0.01889056  0.23537295  0.43606091  0.30855269
 -0.766585   -0.13775047  0.12953293  0.1839

79/79 [==============================] - 0s 1ms/step - loss: 0.3608 - accuracy: 0.9352
Test loss: 0.3608079254627228
Test accuracy: 0.9351999759674072
